In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

# File paths (Hey team please change to your local path 1,2 3)
forecast_path = r"C:\Users\waseem\Desktop\UNSW\Graduation Project\forecastdemand_nsw.csv" #Change path
demand_path = r"C:\Users\waseem\Desktop\UNSW\Graduation Project\totaldemand_nsw.csv" # Change path
temperature_path = r"C:\Users\waseem\Desktop\UNSW\Graduation Project\temperature_nsw.csv" #Change path
target_periodid = 24

# Load datasets

df_forecast = pd.read_csv(forecast_path)
df_demand = pd.read_csv(demand_path)
df_temperature = pd.read_csv(temperature_path)

print(f"Loaded forecast data: {len(df_forecast)} rows")
print(f"Loaded demand data: {len(df_demand)} rows")
print(f"Loaded temperature data: {len(df_temperature)} rows")

# Display samples to check datetime formats
print("\nSample forecast datetime:", df_forecast['DATETIME'].iloc[0] if len(df_forecast) > 0 else "No data")
print("Sample demand datetime:", df_demand['DATETIME'].iloc[0] if len(df_demand) > 0 else "No data")
temp_dt_col = 'date_time' if 'date_time' in df_temperature.columns else 'DATETIME'
print(f"Sample temperature {temp_dt_col}:", df_temperature[temp_dt_col].iloc[0] if len(df_temperature) > 0 else "No data")

# Filter forecast data for PERIODID 24
df_forecast = df_forecast[df_forecast['PERIODID'] == target_periodid]
print(f"Filtered forecast data for PERIODID {target_periodid}: {len(df_forecast)} rows")

# Check if the forecast and demand datetime are already in ISO format (YYYY-MM-DD)
forecast_date_format = "ISO" if '-' in str(df_forecast['DATETIME'].iloc[0]) else "Australian"
demand_date_format = "ISO" if '-' in str(df_demand['DATETIME'].iloc[0]) else "Australian"
print(f"Forecast datetime format appears to be: {forecast_date_format}")
print(f"Demand datetime format appears to be: {demand_date_format}")

# Parse dates - with appropriate handling for existing formats
print("Parsing dates...")

# For forecast data
if forecast_date_format == "ISO":
    # Already in ISO format, just parse
    df_forecast['DATETIME'] = pd.to_datetime(df_forecast['DATETIME'], errors='coerce')
else:
    # Australian format, convert to ISO
    df_forecast['DATETIME'] = pd.to_datetime(df_forecast['DATETIME'], format="%d/%m/%Y %H:%M", errors='coerce')

# For demand data
if demand_date_format == "ISO":
    # Already in ISO format, just parse
    df_demand['DATETIME'] = pd.to_datetime(df_demand['DATETIME'], errors='coerce')
else:
    # Australian format, convert to ISO
    df_demand['DATETIME'] = pd.to_datetime(df_demand['DATETIME'], format="%d/%m/%Y %H:%M", errors='coerce')

# For temperature data
if 'date_time' in df_temperature.columns:
    # First convert from Australian format to datetime
    df_temperature['date_time'] = pd.to_datetime(df_temperature['date_time'], format="%d/%m/%Y %H:%M", errors='coerce')
    # Then create a DATETIME column in the same format as forecast/demand
    df_temperature['DATETIME'] = df_temperature['date_time']
else:
    # Directly parse the DATETIME column
    df_temperature['DATETIME'] = pd.to_datetime(df_temperature['DATETIME'], format="%d/%m/%Y %H:%M", errors='coerce')

# Drop rows with invalid dates
df_forecast = df_forecast.dropna(subset=['DATETIME'])
df_demand = df_demand.dropna(subset=['DATETIME'])
df_temperature = df_temperature.dropna(subset=['DATETIME'])

# Print sample of  dates to verify format consistency
print("\nAfter parsing:")
print("Sample forecast datetime:", df_forecast['DATETIME'].iloc[0] if len(df_forecast) > 0 else "No data")
print("Sample demand datetime:", df_demand['DATETIME'].iloc[0] if len(df_demand) > 0 else "No data")
print("Sample temperature datetime:", df_temperature['DATETIME'].iloc[0] if len(df_temperature) > 0 else "No data")

# Identify temperature column
temp_column = 'temperature' if 'temperature' in df_temperature.columns else 'TEMPERATURE'
print(f"Using temperature column: {temp_column}")

# Merge forecast and demand data
print("Merging forecast with demand data...")
merged_df = pd.merge(
    df_forecast,
    df_demand[['DATETIME', 'TOTALDEMAND', 'REGIONID']],
    on=['DATETIME', 'REGIONID'],
    how='inner'
)
print(f"Merged forecast and demand: {len(merged_df)} rows")

# If merge failed, debug by examining values more closely
if len(merged_df) == 0:
    print("\nDEBUG: Merge failed - examining DATETIME values")

    # Convert all to strings in ISO format for comparison
    df_forecast['DATETIME_STR'] = df_forecast['DATETIME'].dt.strftime('%Y-%m-%d %H:%M:%S')
    df_demand['DATETIME_STR'] = df_demand['DATETIME'].dt.strftime('%Y-%m-%d %H:%M:%S')

    # Print some samples for comparison
    print("\nForecast DATETIME samples:")
    print(df_forecast['DATETIME_STR'].head(5).tolist())
    print("\nDemand DATETIME samples:")
    print(df_demand['DATETIME_STR'].head(5).tolist())

    # Check if there are any exact matches
    forecast_set = set(df_forecast['DATETIME_STR'].tolist())
    demand_set = set(df_demand['DATETIME_STR'].tolist())
    common = forecast_set.intersection(demand_set)
    print(f"\nNumber of common datetime values: {len(common)}")

    # Try a more flexible merge on date only
    print("\nTrying a more flexible merge on date only...")
    df_forecast['DATE'] = df_forecast['DATETIME'].dt.date
    df_demand['DATE'] = df_demand['DATETIME'].dt.date

    date_merged = pd.merge(
        df_forecast,
        df_demand[['DATE', 'TOTALDEMAND', 'REGIONID']],
        on=['DATE', 'REGIONID'],
        how='inner'
    )
    print(f"Date-only merge produced {len(date_merged)} rows")

    if len(date_merged) > 0:
        merged_df = date_merged
        print("Using date-only merge for analysis")
    else:
        print("Analysis cannot continue without matching data")
        exit(1)

# Aggregate temperature by hour to handle multiple readings per hour
print("Aggregating temperature data by hour...")
df_temperature['hour'] = df_temperature['DATETIME'].dt.floor('H')
hourly_temp = df_temperature.groupby('hour')[temp_column].mean().reset_index()
hourly_temp.rename(columns={temp_column: 'TEMPERATURE'}, inplace=True)

# Merge with temperature data
print("Merging with temperature data...")
# Create an hour column in the merged data for joining data
merged_df['hour'] = merged_df['DATETIME'].dt.floor('H')
full_df = pd.merge(
    merged_df,
    hourly_temp,
    on='hour',
    how='left'
)
print(f"Final merged dataset: {len(full_df)} rows")

# If no temperature data was merged, try a different approach
if full_df['TEMPERATURE'].isna().all():
    print("\nDEBUG: Temperature merge failed - trying date-based match")

    # Create date columns
    merged_df['DATE'] = merged_df['DATETIME'].dt.date
    df_temperature['DATE'] = df_temperature['DATETIME'].dt.date

    # Aggregate temperature by date
    daily_temp = df_temperature.groupby('DATE')[temp_column].mean().reset_index()
    daily_temp.rename(columns={temp_column: 'TEMPERATURE'}, inplace=True)

    # Merge on date
    full_df = pd.merge(
        merged_df,
        daily_temp,
        on='DATE',
        how='left'
    )
    print(f"Date-based temperature merge: {len(full_df)} rows with non-null temperature: {full_df['TEMPERATURE'].notna().sum()}")

# Calculate error metrics
full_df['ABS_ERROR'] = abs(full_df['FORECASTDEMAND'] - full_df['TOTALDEMAND'])
full_df['PERC_ERROR'] = 100 * full_df['ABS_ERROR'] / full_df['TOTALDEMAND']

# Overall accuracy metrics
mae = full_df['ABS_ERROR'].mean()
mape = full_df['PERC_ERROR'].mean()
rmse = np.sqrt((full_df['FORECASTDEMAND'] - full_df['TOTALDEMAND']).pow(2).mean())

# Calculate R-squared (coefficient of determination)
ss_total = ((full_df['TOTALDEMAND'] - full_df['TOTALDEMAND'].mean()) ** 2).sum()
ss_residual = ((full_df['TOTALDEMAND'] - full_df['FORECASTDEMAND']) ** 2).sum()
r_squared = 1 - (ss_residual / ss_total)

print("\nOverall Forecast Accuracy Metrics for PERIODID 24:")
print(f"Mean Absolute Error (MAE): {mae:.2f} MW")
print(f"Mean Absolute Percentage Error (MAPE): {mape:.2f}%")
print(f"Root Mean Square Error (RMSE): {rmse:.2f} MW")
print(f"R-squared (R²): {r_squared:.4f}")

# Save the forecast vs actual data regardless of temperature
full_df[['DATETIME', 'REGIONID', 'FORECASTDEMAND', 'TOTALDEMAND', 'ABS_ERROR', 'PERC_ERROR']].to_csv(
    'forecast_vs_actual_periodid24.csv', index=False
)

# Check if we have temperature data before continuing with temperature analysis
if full_df['TEMPERATURE'].notna().sum() > 0:
    # Analyze impact of temperature on forecast accuracy
    print("\nAnalyzing temperature impact on forecast accuracy...")

    # Remove rows with missing temperature values
    full_df_with_temp = full_df.dropna(subset=['TEMPERATURE'])
    print(f"Rows with temperature data: {len(full_df_with_temp)} out of {len(full_df)} total rows")

    # Create temperature bins with specific temperature ranges as requested
    temp_bins = [-10, 0, 5, 10, 15, 20, 25, 30, 35, 40, 50]
    temp_labels = ['<= 0', '0-5', '5-10', '10-15', '15-20', '20-25', '25-30', '30-35', '35-40', '> 40']

    # Create the temperature ranges with specified bins
    full_df_with_temp['TEMP_RANGE'] = pd.cut(
        full_df_with_temp['TEMPERATURE'],
        bins=temp_bins,
        labels=temp_labels,
        include_lowest=True
    )

    # Print information about the created temperature ranges
    print(f"\nUsing specified temperature ranges:")
    temp_range_counts = full_df_with_temp['TEMP_RANGE'].value_counts().sort_index()
    for range_name, count in temp_range_counts.items():
        print(f"  {range_name}: {count} samples")

    # Function to calculate R-squared for a group
    def calculate_r_squared(group):
        if len(group) < 3:
            return np.nan
        ss_total = ((group['TOTALDEMAND'] - group['TOTALDEMAND'].mean()) ** 2).sum()
        if ss_total == 0:
            return np.nan
        ss_residual = ((group['TOTALDEMAND'] - group['FORECASTDEMAND']) ** 2).sum()
        return 1 - (ss_residual / ss_total)

    # Group by temperature range and calculate accuracy metrics
    accuracy_by_temp = full_df_with_temp.groupby('TEMP_RANGE').agg({
        'ABS_ERROR': 'mean',
        'PERC_ERROR': 'mean',
        'TEMPERATURE': 'count'
    }).rename(columns={'ABS_ERROR': 'MAE', 'PERC_ERROR': 'MAPE', 'TEMPERATURE': 'Count'})

    # Calculate R-squared for each temperature range
    r_squared_by_temp = full_df_with_temp.groupby('TEMP_RANGE').apply(calculate_r_squared)
    accuracy_by_temp['R-squared'] = r_squared_by_temp

    print("\nForecast Accuracy by Temperature Range:")
    display(accuracy_by_temp)

    # Non-linear (polynomial) model with both temperature and forecast as predictors
    print("\nFitting nonlinear model with temperature and forecast as predictors...")

    # Create polynomial features for temperature (to capture U-shape)
    from sklearn.preprocessing import PolynomialFeatures
    from sklearn.linear_model import LinearRegression
    from sklearn.metrics import r2_score

    # Prepare the data
    X = full_df_with_temp[['TEMPERATURE', 'FORECASTDEMAND']]
    y = full_df_with_temp['TOTALDEMAND']

    # Create polynomial features for temperature (degree 2 for U-shape)
    poly = PolynomialFeatures(degree=2, include_bias=False)
    X_temp_poly = poly.fit_transform(full_df_with_temp[['TEMPERATURE']])

    # Combine polynomial temperature features with forecast
    X_combined = np.column_stack((X_temp_poly, full_df_with_temp['FORECASTDEMAND']))

    # Fit the model
    model = LinearRegression()
    model.fit(X_combined, y)

    # Make predictions
    y_pred = model.predict(X_combined)

    # Calculate R-squared
    r2_combined = r2_score(y, y_pred)

    print(f"\nNonlinear Model Results (Polynomial Temperature + Forecast):")
    print(f"R-squared: {r2_combined:.4f}")
    print(f"Coefficients:")

    # Get feature names
    feature_names = [f'Temperature', f'Temperature²', 'Forecast']
    for name, coef in zip(feature_names, model.coef_):
        print(f"  {name}: {coef:.6f}")
    print(f"  Intercept: {model.intercept_:.6f}")

    # Individual R-squared values for comparison
    # 1. Temperature only (linear)
    model_temp_linear = LinearRegression()
    model_temp_linear.fit(full_df_with_temp[['TEMPERATURE']], y)
    r2_temp_linear = r2_score(y, model_temp_linear.predict(full_df_with_temp[['TEMPERATURE']]))

    # 2. Temperature only (polynomial)
    model_temp_poly = LinearRegression()
    model_temp_poly.fit(X_temp_poly, y)
    r2_temp_poly = r2_score(y, model_temp_poly.predict(X_temp_poly))

    # 3. Forecast only
    model_forecast = LinearRegression()
    model_forecast.fit(full_df_with_temp[['FORECASTDEMAND']], y)
    r2_forecast = r2_score(y, model_forecast.predict(full_df_with_temp[['FORECASTDEMAND']]))

    print("\nR-squared Comparison:")
    print(f"  Temperature Only (Linear): {r2_temp_linear:.4f}")
    print(f"  Temperature Only (Polynomial): {r2_temp_poly:.4f}")
    print(f"  Forecast Only: {r2_forecast:.4f}")
    print(f"  Combined Model: {r2_combined:.4f}")

    # Visualize the relationship between temperature, forecast and actual demand
    print("\nCreating 3D visualization of Temperature, Forecast, and Actual Demand...")

    # If we have many data points, sample to make the plot clearer
    plot_data = full_df_with_temp
    if len(full_df_with_temp) > 1000:
        plot_data = full_df_with_temp.sample(1000, random_state=42)

    # Create a 3D scatter plot
    from mpl_toolkits.mplot3d import Axes3D

    fig = plt.figure(figsize=(12, 10))
    ax = fig.add_subplot(111, projection='3d')

    scatter = ax.scatter(
        plot_data['TEMPERATURE'],
        plot_data['FORECASTDEMAND'],
        plot_data['TOTALDEMAND'],
        c=plot_data['TOTALDEMAND'],
        cmap='viridis',
        s=50,
        alpha=0.6
    )

    ax.set_xlabel('Temperature (°C)')
    ax.set_ylabel('Forecast Demand (MW)')
    ax.set_zlabel('Actual Demand (MW)')
    ax.set_title(f'3D Relationship: Temperature, Forecast and Actual Demand (PERIODID {target_periodid})')

    # Add a color bar
    cbar = fig.colorbar(scatter, ax=ax, label='Actual Demand (MW)')

    # Save the 3D plot
    plt.tight_layout()
    plt.savefig('3d_relationship_temp_forecast_actual for temp U obin.png')
    plt.close()

    # Save temperature analysis to CSV
    full_df_with_temp.to_csv('forecast_vs_actual_with_temp_periodid24 for temp u o bin .csv', index=False)
    accuracy_by_temp.reset_index().to_csv('accuracy_by_temperature_periodid24 for temp U o bin .csv', index=False)

    # Create visualizations
    print("\nCreating visualizations...")

    # 1. Scatter plot of Forecasted vs Actual Demand colored by temperature
    plt.figure(figsize=(10, 8))
    scatter = plt.scatter(
        full_df_with_temp['TOTALDEMAND'],
        full_df_with_temp['FORECASTDEMAND'],
        c=full_df_with_temp['TEMPERATURE'],
        cmap='coolwarm',
        alpha=0.7
    )
    plt.colorbar(scatter, label='Temperature (°C)')
    plt.plot([full_df_with_temp['TOTALDEMAND'].min(), full_df_with_temp['TOTALDEMAND'].max()],
             [full_df_with_temp['TOTALDEMAND'].min(), full_df_with_temp['TOTALDEMAND'].max()],
             'k--', label='Perfect Forecast')
    plt.title(f'Forecast vs Actual Demand (PERIODID {target_periodid})')
    plt.xlabel('Actual Demand (MW)')
    plt.ylabel('Forecasted Demand (MW)')
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.savefig('forecast_vs_actual_periodid24 fortemp U o bin .png')
    plt.close()

    # 2. Error distribution by temperature range
    plt.figure(figsize=(12, 8))

    # Create boxplot manually to ensure correct ordering
    ranges = full_df_with_temp['TEMP_RANGE'].unique().tolist()
    ranges.sort()  # Sort the ranges

    # Collect data for each range
    box_data = [full_df_with_temp[full_df_with_temp['TEMP_RANGE'] == temp_range]['PERC_ERROR'].values
                for temp_range in ranges]

    # Create the boxplot
    plt.boxplot(box_data, labels=ranges)

    plt.title(f'Forecast Error Distribution by Temperature Range (PERIODID {target_periodid})')
    plt.xlabel('Temperature Range')
    plt.ylabel('Percentage Error (%)')
    plt.xticks(rotation=45)
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.savefig('error_by_temperature_periodid24 fortemp U o bin .png')
    plt.close()

    # 3. Correlation between temperature and forecast error
    if len(full_df_with_temp) > 10:
        plt.figure(figsize=(10, 6))
        plt.scatter(full_df_with_temp['TEMPERATURE'], full_df_with_temp['PERC_ERROR'], alpha=0.5)
        plt.title(f'Temperature vs Forecast Error (PERIODID {target_periodid})')
        plt.xlabel('Temperature (°C)')
        plt.ylabel('Percentage Error (%)')
        plt.grid(True, alpha=0.3)

        # Add trend line
        if len(full_df_with_temp) > 2:  # Need at least 3 points for a trend line
            z = np.polyfit(full_df_with_temp['TEMPERATURE'], full_df_with_temp['PERC_ERROR'], 1)
            p = np.poly1d(z)
            temp_range = np.linspace(full_df_with_temp['TEMPERATURE'].min(), full_df_with_temp['TEMPERATURE'].max(), 100)
            plt.plot(temp_range, p(temp_range), "r--", linewidth=2)

            corr = full_df_with_temp['TEMPERATURE'].corr(full_df_with_temp['PERC_ERROR'])
            plt.annotate(f'Correlation: {corr:.4f}',
                        xy=(0.05, 0.95),
                        xycoords='axes fraction',
                        bbox=dict(boxstyle="round,pad=0.3", fc="white", ec="gray", alpha=0.8))

        plt.tight_layout()
        plt.savefig('temperature_vs_error_periodid24 fortemp U Obin.png')
        plt.close()

    # 4. R-squared by temperature range
    if 'R-squared' in accuracy_by_temp.columns and not accuracy_by_temp['R-squared'].isna().all():
        plt.figure(figsize=(12, 6))

        # Convert index to list for proper ordering in the plot
        ranges = accuracy_by_temp.index.tolist()

        # Plot the bar chart
        plt.bar(range(len(ranges)), accuracy_by_temp['R-squared'])

        # Set the tick positions and labels
        plt.xticks(range(len(ranges)), ranges, rotation=45)

        plt.title(f'R-squared by Temperature Range (PERIODID {target_periodid})')
        plt.xlabel('Temperature Range')
        plt.ylabel('R-squared (R²)')
        plt.ylim(0, 1)  # R-squared is typically between 0 and 1
        plt.grid(True, alpha=0.3)
        plt.tight_layout()
        plt.savefig('r_squared_by_temperature_periodid24 fortemp U Obin.png')
        plt.close()
else:
    print("\nNo temperature data was successfully matched with forecast/demand data.")
    print("Skipping temperature-related analysis.")


# Get a sample period for better visualization (most recent 14 days)
full_df_sorted = full_df.sort_values('DATETIME')
sample_period = full_df_sorted.tail(min(24*14, len(full_df_sorted)))

plt.figure(figsize=(14, 7))
plt.plot(sample_period['DATETIME'], sample_period['TOTALDEMAND'], 'b-', label='Actual Demand')
plt.plot(sample_period['DATETIME'], sample_period['FORECASTDEMAND'], 'r--', label='Forecast Demand')
plt.title(f'Forecast vs Actual Demand Over Time (PERIODID {target_periodid})')
plt.xlabel('Date')
plt.ylabel('Demand (MW)')
plt.grid(True, alpha=0.3)
plt.legend()
plt.tight_layout()
plt.savefig('time_series_forecast_actual_periodid24 fortemp U Obin.png')
plt.close()



Loading datasets...


C:\Users\waseem\AppData\Local\Temp\ipykernel_17264\1254910683.py:20: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  df_temperature = pd.read_csv(temperature_path)


Loaded forecast data: 10906019 rows
Loaded demand data: 196513 rows
Loaded temperature data: 220326 rows

Sample forecast datetime: 2010-01-01 00:00:00
Sample demand datetime: 1/01/2010 0:00
Sample temperature DATETIME: 1/01/2010 0:00
Filtered forecast data for PERIODID 24: 196505 rows
Forecast datetime format appears to be: ISO
Demand datetime format appears to be: Australian
Parsing dates...

After parsing:
Sample forecast datetime: 2010-01-01 00:00:00
Sample demand datetime: 2010-01-01 00:00:00
Sample temperature datetime: 2010-01-01 00:00:00
Using temperature column: TEMPERATURE
Merging forecast with demand data...
Merged forecast and demand: 196505 rows
Aggregating temperature data by hour...
Merging with temperature data...
Final merged dataset: 196505 rows


C:\Users\waseem\AppData\Local\Temp\ipykernel_17264\1254910683.py:138: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  df_temperature['hour'] = df_temperature['DATETIME'].dt.floor('H')
C:\Users\waseem\AppData\Local\Temp\ipykernel_17264\1254910683.py:145: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  merged_df['hour'] = merged_df['DATETIME'].dt.floor('H')



Overall Forecast Accuracy Metrics for PERIODID 24:
Mean Absolute Error (MAE): 169.09 MW
Mean Absolute Percentage Error (MAPE): 2.05%
Root Mean Square Error (RMSE): 234.87 MW
R-squared (R²): 0.9673

Analyzing temperature impact on forecast accuracy...
Rows with temperature data: 196141 out of 196505 total rows

Using specified temperature ranges:
  <= 0: 16 samples
  0-5: 2710 samples
  5-10: 19550 samples
  10-15: 42344 samples
  15-20: 62603 samples
  20-25: 51527 samples
  25-30: 14269 samples
  30-35: 2568 samples
  35-40: 484 samples
  > 40: 70 samples

Forecast Accuracy by Temperature Range:


C:\Users\waseem\AppData\Local\Temp\ipykernel_17264\1254910683.py:214: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  full_df_with_temp['TEMP_RANGE'] = pd.cut(
C:\Users\waseem\AppData\Local\Temp\ipykernel_17264\1254910683.py:238: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  accuracy_by_temp = full_df_with_temp.groupby('TEMP_RANGE').agg({
C:\Users\waseem\AppData\Local\Temp\ipykernel_17264\1254910683.py:245: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current

,MAE,MAPE,Count,R-squared
TEMP_RANGE,,,,
<= 0,162.318750,2.090544,16,0.960227
0-5,132.277295,1.653575,2710,0.975757
5-10,137.099084,1.678702,19550,0.983940
10-15,160.561842,1.919153,42344,0.977542
15-20,150.157787,1.936071,62603,0.963830
20-25,175.763657,2.162140,51527,0.941278
25-30,261.557903,2.840032,14269,0.930041
30-35,348.152960,3.455496,2568,0.912743
35-40,441.745868,3.869580,484,0.810509



Fitting nonlinear model with temperature and forecast as predictors...

Nonlinear Model Results (Polynomial Temperature + Forecast):
R-squared: 0.9687
Coefficients:
  Temperature: -25.346849
  Temperature²: 0.769912
  Forecast: 0.969277
  Intercept: 438.465014

R-squared Comparison:
  Temperature Only (Linear): 0.0222
  Temperature Only (Polynomial): 0.0971
  Forecast Only: 0.9679
  Combined Model: 0.9687

Creating 3D visualization of Temperature, Forecast, and Actual Demand...


PermissionError: [Errno 13] Permission denied: 'accuracy_by_temperature_periodid24 for temp U o bin .csv'